In [22]:
import threading
from datetime import datetime

#### Thread Local Variable

In [4]:
ThreadLocalVar = threading.local()
ThreadLocalVar.a = 5
ThreadLocalVar.b = 10
print(a, b)

NameError: name 'a' is not defined

In [5]:
ThreadLocalVar

#### Lock and RLock

In [28]:
lock_access = threading.Lock()# Timeout in seconds, blocking default = True Blocks other threads
lock_Reentrant = threading.RLock(blocking=True, timeout=10)

In [29]:
def func_a(message="", times=10):
    print(times)
    #lock_access.acquire()
    lock_Reentrant.acquire()
    for count in range(0,times): 
        print(message, count)
        print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    lock_Reentrant.release()
    #lock_access.release()

In [38]:
#Threading.Thread(group=None, target=None, name=None, args=(), kwargs={}, *, daemon=None)
x1 = threading.Thread(target=func_a,kwargs=dict(message='Hello', times=10))
x2 = threading.Thread(target=func_a,kwargs=dict(message='Welcome', times=20))
x3 = threading.Thread(target=func_a,kwargs=dict(message='Hi', times=12))


##### thread.join() waits for current thread to complete and suspends other threads by acquiring the lock and when complete notifies other blocked threads to continue

In [39]:
x1.start()
x2.start()
x3.start()
x3.join()

10
Hello 0
2020-11-30 00:49:07
Hello 1
2020-11-30 00:49:07
Hello 202
2020-11-30 00:49:0712

Hello 3
2020-11-30 00:49:07

Hello 4
2020-11-30 00:49:07
Hello 5
2020-11-30 00:49:07
Hello 6
2020-11-30 00:49:07
Hello 7
2020-11-30 00:49:07
Hello 8
2020-11-30 00:49:07
Hello 9
2020-11-30 00:49:07
Hi 0
2020-11-30 00:49:07
Hi 1
2020-11-30 00:49:07
Hi 2
2020-11-30 00:49:07
Hi 3
2020-11-30 00:49:07
Hi 4
2020-11-30 00:49:07
Hi 5
2020-11-30 00:49:07
Hi 6
2020-11-30 00:49:07
Hi 7
2020-11-30 00:49:07
Hi 8
2020-11-30 00:49:07
Hi 9
2020-11-30 00:49:07
Hi 10
2020-11-30 00:49:07
Hi 11
2020-11-30 00:49:07
Welcome 0
2020-11-30 00:49:07
Welcome 1
2020-11-30 00:49:07
Welcome 2
2020-11-30 00:49:07
Welcome 3
2020-11-30 00:49:07
Welcome 4
2020-11-30 00:49:07
Welcome 5
2020-11-30 00:49:07
Welcome 6
2020-11-30 00:49:07
Welcome 7
2020-11-30 00:49:07
Welcome 8
2020-11-30 00:49:07
Welcome 9
2020-11-30 00:49:07
Welcome 10
2020-11-30 00:49:07
Welcome 11
2020-11-30 00:49:07
Welcome 12
2020-11-30 00:49:07
Welcome 13
2020-

#### Condition Objects

In [42]:
import threading
import time
import logging

logging.basicConfig(level=logging.DEBUG,format='(%(threadName)-9s) %(message)s',)

def consumer(cv):
    logging.debug('Consumer thread started ...')
    with cv:
        logging.debug('Consumer waiting ...')
        cv.wait()
        logging.debug('Consumer consumed the resource')

def producer(cv):
    logging.debug('Producer thread started ...')
    with cv:
        logging.debug('Making resource available')
        logging.debug('Notifying to all consumers')
        cv.notifyAll()

if __name__ == '__main__':
    condition = threading.Condition()
    cs1 = threading.Thread(name='consumer1', target=consumer, args=(condition,))
    cs2 = threading.Thread(name='consumer2', target=consumer, args=(condition,))
    pd = threading.Thread(name='producer', target=producer, args=(condition,))

    cs1.start()
    time.sleep(2)
    cs2.start()
    time.sleep(2)
    pd.start()

(consumer1) Consumer thread started ...
(consumer1) Consumer waiting ...
(consumer2) Consumer thread started ...
(consumer2) Consumer waiting ...
(producer ) Producer thread started ...
(producer ) Making resource available
(producer ) Notifying to all consumers
(consumer2) Consumer consumed the resource
(consumer1) Consumer consumed the resource


#### Semaphore 

In [44]:
import logging
import random
import threading
import time

logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s (%(threadName)-2s) %(message)s',
                    )

class ActivePool(object):
    def __init__(self):
        super(ActivePool, self).__init__()
        self.active = []
        self.lock = threading.Lock()
    def makeActive(self, name):
        with self.lock:
            self.active.append(name)
            logging.debug('Running: %s', self.active)
    def makeInactive(self, name):
        with self.lock:
            self.active.remove(name)
            logging.debug('Running: %s', self.active)

def worker(s, pool):
    logging.debug('Waiting to join the pool')
    with s:
        name = threading.currentThread().getName()
        pool.makeActive(name)
        time.sleep(0.1)
        pool.makeInactive(name)

pool = ActivePool()
s = threading.Semaphore(2)
for i in range(4):
    t = threading.Thread(target=worker, name=str(i), args=(s, pool))
    t.start()

(0        ) Waiting to join the pool
(0        ) Running: ['0']
(1        ) Waiting to join the pool
(2        ) Waiting to join the pool
(3        ) Waiting to join the pool
(1        ) Running: ['0', '1']
(0        ) Running: ['1']
(2        ) Running: ['1', '2']
(1        ) Running: ['2']
(3        ) Running: ['2', '3']
(2        ) Running: ['3']
(3        ) Running: []


#### Events

In [43]:
import logging
import threading
import time

logging.basicConfig(level=logging.DEBUG,
                    format='(%(threadName)-10s) %(message)s',
                    )
                    
def wait_for_event(e):
    """Wait for the event to be set before doing anything"""
    logging.debug('wait_for_event starting')
    event_is_set = e.wait()
    logging.debug('event set: %s', event_is_set)

def wait_for_event_timeout(e, t):
    """Wait t seconds and then timeout"""
    while not e.isSet():
        logging.debug('wait_for_event_timeout starting')
        event_is_set = e.wait(t)
        logging.debug('event set: %s', event_is_set)
        if event_is_set:
            logging.debug('processing event')
        else:
            logging.debug('doing other work')


e = threading.Event()
t1 = threading.Thread(name='block', 
                      target=wait_for_event,
                      args=(e,))
t1.start()

t2 = threading.Thread(name='non-block', 
                      target=wait_for_event_timeout, 
                      args=(e, 2))
t2.start()

logging.debug('Waiting before calling Event.set()')
time.sleep(3)
e.set()
logging.debug('Event is set')

(block    ) wait_for_event starting
(non-block) wait_for_event_timeout starting
(MainThread) Waiting before calling Event.set()
(non-block) event set: False
(non-block) doing other work
(non-block) wait_for_event_timeout starting
(MainThread) Event is set
(block    ) event set: True
(non-block) event set: True
(non-block) processing event
